不動産価格予測

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mljar-supervised >> /dev/null

In [3]:
!pip install optuna xfeat japanize-matplotlib >> /dev/null

In [4]:
# パスの設定
My_PATH = "/content/drive/MyDrive/property_values_prediction"
%cd {My_PATH}

/content/drive/MyDrive/property_values_prediction


In [5]:
# data.zipファイルの解凍
# 古いdataのディレクトリ削除
!rm -rf /content/drive/MyDrive/property_values_prediction/data/　　>> /dev/null

import zipfile
with zipfile.ZipFile('data.zip', 'r')as f:
    f.extractall('./data')

# train.zipの解凍
# 古いtrainディレクトリの削除
!rm -rf /content/drive/MyDrive/property_values_prediction/data/train/ >> /dev/null

with zipfile.ZipFile('data/train.zip', 'r')as f:
    f.extractall('./data/')

In [6]:
import datetime
import os

now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d-%H-%M")
dir_for_output = "./output/" + current_time

os.makedirs(dir_for_output, exist_ok=True)

In [7]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import supervised
from supervised.automl import AutoML
import pandas as pd
import seaborn as sns
sns.set(font="IPAexGothic")

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import (
    SelectCategorical,
    LabelEncoder,
    LambdaEncoder,
    Pipeline,
    ConcatCombination,
    SelectNumerical,
    ArithmeticCombinations,
    TargetEncoder,
    aggregation,
    GBDTFeatureSelector,
    GBDTFeatureExplorer,
)

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [8]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(moyori)
    return moyori

def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

コンペデータ作成（利用カラム、目的変数など決める）

In [9]:
BASE_PATH = './data/'
RANDOM_STATE = 10

In [10]:
paths = glob(BASE_PATH + 'train/*')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)
test_df = pd.read_csv(BASE_PATH + 'test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.



In [11]:
train_df.isnull().sum()

ID                   0
種類                   0
地域              677392
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                660
最寄駅：名称            2672
最寄駅：距離（分）        23098
間取り              23661
面積（㎡）                0
土地の形状           677392
間口              677392
延床面積（㎡）         677392
建築年              19622
建物の構造            16486
用途               58246
今後の利用目的         364049
前面道路：方位         677392
前面道路：種類         677392
前面道路：幅員（ｍ）      677392
都市計画             19221
建ぺい率（％）          23345
容積率（％）           23345
取引時点                 0
改装               61966
取引の事情等          658854
取引価格（総額）_log         0
dtype: int64

In [12]:
sub_df = pd.read_csv(BASE_PATH + 'sample_submission.csv')

In [13]:
ID = 'ID'
TARGET = '取引価格（総額）_log'

In [14]:
df = pd.concat([train_df, test_df])

In [15]:
# 解析対象外のカラムを指定
rm_cols = []
rm_cols += ['市区町村コード']

# ユニーク数が1以下のカラムを解析対象外のカラムとして指定
for i,v in df.nunique().iteritems():
    if v <= 1:
        rm_cols.append(i)

rm_cols

['市区町村コード',
 '種類',
 '地域',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）']

In [16]:
# テストデータの目的変数が無いため、np.nan で埋める
test_df[TARGET] = np.nan

# 解析対象外のカラムを削除する
train_df.drop(rm_cols, axis=1, inplace=True)
test_df.drop(rm_cols, axis=1, inplace=True)

# 解析対象に使用するDataFrameを作成
df = pd.concat([train_df, test_df])

# 取引時点(売買契約/四半期単位)でソートし、indexを振り直す
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(700543, 19)

In [17]:
# 取引時点(売買契約/四半期単位)のユニークを確認
df['取引時点'].unique()

array(['2005年第３四半期', '2005年第４四半期', '2006年第１四半期', '2006年第２四半期',
       '2006年第３四半期', '2006年第４四半期', '2007年第１四半期', '2007年第２四半期',
       '2007年第３四半期', '2007年第４四半期', '2008年第１四半期', '2008年第２四半期',
       '2008年第３四半期', '2008年第４四半期', '2009年第１四半期', '2009年第２四半期',
       '2009年第３四半期', '2009年第４四半期', '2010年第１四半期', '2010年第２四半期',
       '2010年第３四半期', '2010年第４四半期', '2011年第１四半期', '2011年第２四半期',
       '2011年第３四半期', '2011年第４四半期', '2012年第１四半期', '2012年第２四半期',
       '2012年第３四半期', '2012年第４四半期', '2013年第１四半期', '2013年第２四半期',
       '2013年第３四半期', '2013年第４四半期', '2014年第１四半期', '2014年第２四半期',
       '2014年第３四半期', '2014年第４四半期', '2015年第１四半期', '2015年第２四半期',
       '2015年第３四半期', '2015年第４四半期', '2016年第１四半期', '2016年第２四半期',
       '2016年第３四半期', '2016年第４四半期', '2017年第１四半期', '2017年第２四半期',
       '2017年第３四半期', '2017年第４四半期', '2018年第１四半期', '2018年第２四半期',
       '2018年第３四半期', '2018年第４四半期', '2019年第１四半期', '2019年第２四半期',
       '2019年第３四半期', '2019年第４四半期', '2020年第１四半期', '2020年第２四半期',
       '2020年第３四半期', '2020年第４四半期', '2021年第１四半期'], dtype

In [18]:
val_min_idx = min(df[df['取引時点'].str.contains('2020年第２四半期|2020年第３四半期', regex=True)].index)
test_min_idx = min(df[df['取引時点'].str.contains('2020年第４四半期|2021年第１四半期', regex=True)].index)
val_min_idx, test_min_idx

(652493, 677392)

In [19]:
set(df.iloc[val_min_idx:test_min_idx, :]['取引時点'].values)

{'2020年第２四半期', '2020年第３四半期'}

特徴量生成

In [20]:
# 取引時点(売買契約/四半期単位)を順序尺度に変換
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)
df.tail()

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc
700538,23000726,愛知県,名古屋市千種区,向陽,池下,6,１ＬＤＫ,60,昭和49年,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2021年第１四半期,改装済,NaN,NaN,62
700539,23000787,愛知県,名古屋市千種区,桜が丘,星ケ丘(愛知),11,３ＬＤＫ,80,平成27年,ＲＣ,住宅,住宅,第１種住居地域,60.0,200.0,2021年第１四半期,未改装,NaN,NaN,62
700540,13014303,東京都,新宿区,中落合,落合南長崎,8,１Ｋ,20,平成15年,ＲＣ,NaN,住宅,近隣商業地域,80.0,400.0,2021年第１四半期,未改装,NaN,NaN,62
700541,23001017,愛知県,名古屋市千種区,振甫町,覚王山,15,３ＬＤＫ,70,平成26年,ＲＣ,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2021年第１四半期,未改装,NaN,NaN,62
700542,12003545,千葉県,千葉市花見川区,幕張町,幕張,16,３ＬＤＫ,80,昭和57年,ＳＲＣ,住宅,住宅,市街化調整区域,60.0,200.0,2021年第１四半期,NaN,NaN,NaN,62


In [21]:
# 都道府県別の取引時点(売買契約/四半期単位)別の平均売買件数を算出
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836439,
 '京都府': 7.155935848555253,
 '佐賀県': 7.009937258263509,
 '兵庫県': 7.157742870483196,
 '北海道': 6.965967793677859,
 '千葉県': 7.155825324964995,
 '和歌山県': 7.022690210815773,
 '埼玉県': 7.1697442713966355,
 '大分県': 6.911537826573079,
 '大阪府': 7.158352973821201,
 '奈良県': 7.056884610234688,
 '宮城県': 7.0395442538048805,
 '宮崎県': 6.958678653711235,
 '富山県': 7.0469200713870155,
 '山口県': 7.045342730349803,
 '山形県': 7.087990602814902,
 '山梨県': 6.848839888686708,
 '岐阜県': 7.078095913486453,
 '岡山県': 7.03809073492747,
 '岩手県': 6.925002679060599,
 '島根県': 7.165376214700312,
 '広島県': 7.093449021655064,
 '徳島県': 6.912051981629745,
 '愛媛県': 6.966433011596662,
 '愛知県': 7.107301969238601,
 '新潟県': 6.913835156235044,
 '東京都': 7.377929204641064,
 '栃木県': 6.950636958071011,
 '沖縄県': 7.160046604428336,
 '滋賀県': 7.16602695816839,
 '熊本県': 6.970182251843263,
 '石川県': 6.929363348877185,
 '神奈川県': 7.2581182672098175,
 '福井県': 6.979575815840195,
 '福岡県': 6.992264905042226,
 '福島県': 6.9743805791401785,
 '秋田県': 6.9413137471126

In [22]:
# 都道府県別の取引時点(売買契約/四半期単位)別平均売買件数を追加
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)
df.tail()

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te
700538,23000726,愛知県,名古屋市千種区,向陽,池下,6,１ＬＤＫ,60,昭和49年,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2021年第１四半期,改装済,NaN,NaN,62,7.124361
700539,23000787,愛知県,名古屋市千種区,桜が丘,星ケ丘(愛知),11,３ＬＤＫ,80,平成27年,ＲＣ,住宅,住宅,第１種住居地域,60.0,200.0,2021年第１四半期,未改装,NaN,NaN,62,7.124361
700540,13014303,東京都,新宿区,中落合,落合南長崎,8,１Ｋ,20,平成15年,ＲＣ,NaN,住宅,近隣商業地域,80.0,400.0,2021年第１四半期,未改装,NaN,NaN,62,7.396885
700541,23001017,愛知県,名古屋市千種区,振甫町,覚王山,15,３ＬＤＫ,70,平成26年,ＲＣ,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2021年第１四半期,未改装,NaN,NaN,62,7.124361
700542,12003545,千葉県,千葉市花見川区,幕張町,幕張,16,３ＬＤＫ,80,昭和57年,ＳＲＣ,住宅,住宅,市街化調整区域,60.0,200.0,2021年第１四半期,NaN,NaN,NaN,62,7.161930


In [23]:
df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2021-int(x[:4]))
df.drop(['取引時点'], axis=1, inplace=True)
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))
df.head()

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前
0,27232131,大阪府,大阪市旭区,新森,森小路,6.0,１ＬＤＫ,50,1974.0,ＲＣ,住宅,NaN,準工業地域,80.0,200.0,改装済,NaN,7.041393,0,0.0,16
1,14289301,神奈川県,川崎市高津区,千年新町,武蔵新城,7.0,３ＬＤＫ,60,1992.0,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,改装済,NaN,7.361728,0,0.0,16
2,13417040,東京都,葛飾区,宝町,お花茶屋,NaN,２ＬＤＫ,60,1993.0,ＲＣ,住宅,NaN,準工業地域,60.0,200.0,改装済,NaN,7.255273,0,0.0,16
3,14267697,神奈川県,横浜市青葉区,あざみ野,あざみ野,NaN,３ＬＤＫ,85,1982.0,NaN,住宅,NaN,NaN,NaN,NaN,NaN,NaN,7.556303,0,0.0,16
4,13386603,東京都,練馬区,豊玉北,練馬,5.0,１Ｋ,20,2005.0,ＲＣ,住宅,NaN,商業地域,80.0,500.0,未改装,NaN,7.278754,0,0.0,16


In [24]:
# 数値データのみを抽出(xfeat)
num_df = SelectNumerical().fit_transform(df)
num_df.head(2)

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前
0,27232131,6.0,50,1974.0,80.0,200.0,7.041393,0,0.0,16
1,14289301,7.0,60,1992.0,60.0,200.0,7.361728,0,0.0,16


In [25]:
# カテゴリデータのラベルエンコーディング
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head(2)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等
0,0,0,0,0,0,0,0,-1,0,0,-1
1,1,1,1,1,1,0,0,-1,1,0,-1


In [26]:
# 面積×容積率を計算
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"], 
            drop_origin=True, 
            operator="*", 
            r=2,
        ),
    ]
)

num_comb_df = encoder.fit_transform(df)/100
num_comb_df.head(2)

,面積（㎡）容積率（％）_combi
0,100.0
1,120.0


In [27]:
agg_dfs = []

def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(
        df,
        group_key=group_col,
        group_values=['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
        agg_methods=['count', 'mean', 'min', 'max', 'median', 'std'],
        )

    return agg_df[agg_cols]

group_col = '市区町村名'
agg_dfs.append(get_agg_df(df, group_col))
agg_dfs[0].head(2)

,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名,agg_median_最寄駅：距離（分）_grpby_市区町村名,agg_median_面積（㎡）_grpby_市区町村名,agg_median_建ぺい率（％）_grpby_市区町村名,agg_median_容積率（％）_grpby_市区町村名,agg_std_最寄駅：距離（分）_grpby_市区町村名,agg_std_面積（㎡）_grpby_市区町村名,agg_std_建ぺい率（％）_grpby_市区町村名,agg_std_容積率（％）_grpby_市区町村名
0,789,808,806,806,8.206591,59.814356,70.397022,236.228288,0.0,15,60.0,200.0,23.0,115,80.0,400.0,8.0,60.0,80.0,200.0,4.327536,16.170642,9.998320,61.473966
1,2945,3089,2872,2872,13.020713,59.043380,61.922006,212.580084,0.0,15,40.0,80.0,105.0,550,80.0,1100.0,10.0,65.0,60.0,200.0,10.878025,23.045336,6.448643,56.522903


In [28]:
feat_df = pd.concat([num_df,le_df,num_comb_df]+agg_dfs, axis=1)
print(feat_df.shape)

(700543, 46)


In [29]:
feat_df.dtypes

ID                                    int64
最寄駅：距離（分）                           float64
面積（㎡）                                 int64
建築年                                 float64
建ぺい率（％）                             float64
容積率（％）                              float64
取引価格（総額）_log                        float64
取引時点_enc                              int64
都道府県名_te                            float64
取引時点_何年前                              int64
都道府県名                                 int64
市区町村名                                 int64
地区名                                   int64
最寄駅：名称                                int64
間取り                                   int64
建物の構造                                 int64
用途                                    int64
今後の利用目的                               int64
都市計画                                  int64
改装                                    int64
取引の事情等                                int64
面積（㎡）容積率（％）_combi                   float64
agg_count_最寄駅：距離（分）_grpby_市区町村名 

モデル構築

In [30]:
train_df = feat_df.iloc[:test_min_idx, :]
test_df = feat_df.iloc[test_min_idx:, :]
print(train_df.shape, test_df.shape)

(677392, 46) (23151, 46)


In [31]:
train_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名,agg_median_最寄駅：距離（分）_grpby_市区町村名,agg_median_面積（㎡）_grpby_市区町村名,agg_median_建ぺい率（％）_grpby_市区町村名,agg_median_容積率（％）_grpby_市区町村名,agg_std_最寄駅：距離（分）_grpby_市区町村名,agg_std_面積（㎡）_grpby_市区町村名,agg_std_建ぺい率（％）_grpby_市区町村名,agg_std_容積率（％）_grpby_市区町村名
0,27232131,6.0,50,1974.0,80.0,200.0,7.041393,0,0.0,16,0,0,0,0,0,0,0,-1,0,0,-1,100.0,789,808,806,806,8.206591,59.814356,70.397022,236.228288,0.0,15,60.0,200.0,23.0,115,80.0,400.0,8.0,60.0,80.0,200.0,4.327536,16.170642,9.998320,61.473966
1,14289301,7.0,60,1992.0,60.0,200.0,7.361728,0,0.0,16,1,1,1,1,1,0,0,-1,1,0,-1,120.0,2945,3089,2872,2872,13.020713,59.043380,61.922006,212.580084,0.0,15,40.0,80.0,105.0,550,80.0,1100.0,10.0,65.0,60.0,200.0,10.878025,23.045336,6.448643,56.522903
2,13417040,NaN,60,1993.0,60.0,200.0,7.255273,0,0.0,16,2,2,2,2,2,0,0,-1,0,0,-1,120.0,4605,4893,4812,4812,9.963084,50.796035,64.170823,279.640482,0.0,15,40.0,80.0,45.0,270,80.0,600.0,9.0,55.0,60.0,200.0,5.964632,19.976092,8.379281,105.887072
3,14267697,NaN,85,1982.0,NaN,NaN,7.556303,0,0.0,16,1,3,3,3,1,-1,0,-1,-1,-1,-1,NaN,3465,3675,3375,3375,13.894372,67.380952,59.454815,170.755556,0.0,15,30.0,60.0,120.0,210,80.0,500.0,10.0,70.0,60.0,150.0,11.784773,16.498517,8.659345,60.240019
4,13386603,5.0,20,2005.0,80.0,500.0,7.278754,0,0.0,16,2,4,4,4,3,0,0,-1,2,1,-1,100.0,8377,8625,8526,8526,8.184075,46.059710,66.272578,274.364297,0.0,10,30.0,80.0,75.0,220,80.0,600.0,7.0,50.0,60.0,200.0,5.571003,23.541560,10.329144,111.651271


In [32]:
test_df = test_df.fillna(0)
test_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名,agg_median_最寄駅：距離（分）_grpby_市区町村名,agg_median_面積（㎡）_grpby_市区町村名,agg_median_建ぺい率（％）_grpby_市区町村名,agg_median_容積率（％）_grpby_市区町村名,agg_std_最寄駅：距離（分）_grpby_市区町村名,agg_std_面積（㎡）_grpby_市区町村名,agg_std_建ぺい率（％）_grpby_市区町村名,agg_std_容積率（％）_grpby_市区町村名
677392,14068534,11.0,60,1996.0,60.0,200.0,0.0,61,7.268224,1,1,251,5181,1999,2,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,10.0,60.0,60.0,200.0,8.036677,17.062655,8.897978,92.506284
677393,14068533,24.0,70,1998.0,50.0,100.0,0.0,61,7.268224,1,1,251,5181,1999,1,0,0,0,9,0,-1,70.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,10.0,60.0,60.0,200.0,8.036677,17.062655,8.897978,92.506284
677394,14068532,11.0,60,1996.0,60.0,200.0,0.0,61,7.268224,1,1,251,5181,1999,1,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,10.0,60.0,60.0,200.0,8.036677,17.062655,8.897978,92.506284
677395,14068420,21.0,105,1969.0,60.0,200.0,0.0,61,7.268224,1,1,251,5018,1702,19,0,0,0,1,0,-1,210.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,10.0,60.0,60.0,200.0,8.036677,17.062655,8.897978,92.506284
677396,14068219,21.0,65,1979.0,60.0,200.0,0.0,61,7.268224,1,1,251,4480,1544,1,0,-1,0,7,1,-1,130.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,10.0,60.0,60.0,200.0,8.036677,17.062655,8.897978,92.506284


In [33]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

In [34]:
cat_cols = list(le_df.columns) + ['取引時点_enc']
cat_cols

['都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '間取り',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '都市計画',
 '改装',
 '取引の事情等',
 '取引時点_enc']

In [35]:
train_x = train_df[feat_cols]
train_y = train_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

In [36]:
# __init__(
    # results_path=None,
    # total_time_limit=60 * 60,
    # mode='Explain',
    # ml_task='auto',
    # model_time_limit=None,
    # algorithms='auto',
    # train_ensemble=True,
    # stack_models='auto',
    # eval_metric='auto',
    # validation_strategy='auto',
    # explain_level='auto',
    # golden_features='auto',
    # features_selection='auto',
    # start_random_models='auto',
    # hill_climbing_steps='auto',
    # top_models_to_improve='auto',
    # boost_on_errors='auto',
    # kmeans_features='auto',
    # mix_encoding='auto',
    # max_single_prediction_time=None,
    # optuna_time_budget=None,
    # optuna_init_params={},
    # optuna_verbose=True,
    # n_jobs=-1,
    # verbose=1,
    # random_state=1234
    # )

In [37]:
%%time
# Optuna mode
automl_optuna = AutoML(
    mode="Optuna",
    total_time_limit=60 * 60,
    optuna_time_budget=60 * 45,
    algorithms=["LightGBM", "Xgboost", "CatBoost"],
    eval_metric='mae',
    random_state=RANDOM_STATE
    )
automl_optuna.fit(train_x, train_y)

AutoML directory: AutoML_1
Expected computing time:
Time for tuning with Optuna: len(algorithms) * optuna_time_budget = 8100 seconds
There is no time limit for ML model training after Optuna tuning (total_time_limit parameter is ignored).
The task is regression with evaluation metric mae
AutoML will use algorithms: ['LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
Optuna optimizes LightGBM with time budget 2700 seconds eval_metric mae (minimize)


[I 2021-12-01 14:12:46,720] A new study created in memory with name: no-name-0949962e-592d-414a-80b3-568953c83543
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning:

'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.

[I 2021-12-01 14:16:49,787] Trial 0 finished with value: 0.24961745674933639 and parameters: {'learning_rate': 0.0125, 'num_leaves': 1022, 'lambda_l1': 1.0547992438188775e-06, 'lambda_l2': 6.061300044367956e-07, 'feature_fraction': 0.8323715061445782, 'bagging_fraction': 0.41837758728488317, 'bagging_freq': 1, 'min_data_in_leaf': 69, 'extra_trees': True}. Best is trial 0 with value: 0.24961745674933639

1_Optuna_LightGBM mae 0.074328 trained in 6454.75 seconds
Optuna optimizes Xgboost with time budget 2700 seconds eval_metric mae (minimize)


[I 2021-12-01 16:48:06,416] A new study created in memory with name: no-name-924627e0-d6a8-433b-bc10-b8db25b5f819
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:108: UserWarning:

ntree_limit is deprecated, use `iteration_range` or model slicing instead.

[I 2021-12-01 16:50:57,071] Trial 0 finished with value: 0.2488820383467444 and parameters: {'eta': 0.0125, 'max_depth': 7, 'lambda': 1.0547992438188775e-06, 'alpha': 6.061300044367956e-07, 'colsample_bytree': 0.8323715061445782, 'subsample': 0.41837758728488317, 'min_child_weight': 9}. Best is trial 0 with value: 0.2488820383467444.
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:108: UserWarning:

ntree_limit is deprecated, use `iteration_range` or model slicing instead.

[I 2021-12-01 16:54:21,697] Trial 1 finished with value: 0.22153934406232928 and parameters: {'eta': 0.025, 'max_depth': 10, 'lambda': 0.003256376421394008, 'alpha': 0.03131827670437561, 'colsample_bytree': 0.5043132506382039, 'subsample': 0.94244189

Data has no positive values, and therefore can not be log-scaled.
2_Optuna_Xgboost mae 0.075588 trained in 3756.87 seconds
Optuna optimizes CatBoost with time budget 2700 seconds eval_metric mae (minimize)


[I 2021-12-01 18:38:41,340] A new study created in memory with name: no-name-8fa3c672-9c8d-4e51-b766-1ccc53573421
[I 2021-12-01 18:41:29,415] Trial 0 finished with value: 0.2443137805557189 and parameters: {'learning_rate': 0.05, 'depth': 7, 'l2_leaf_reg': 4.985120272324674, 'random_strength': 2.24796646306051, 'rsm': 0.27825657828366157, 'min_data_in_leaf': 77}. Best is trial 0 with value: 0.2443137805557189.
[I 2021-12-01 18:46:11,539] Trial 1 finished with value: 0.21752383625889085 and parameters: {'learning_rate': 0.2, 'depth': 9, 'l2_leaf_reg': 0.039582268452511726, 'random_strength': 5.121922638735843, 'rsm': 0.8313588654869022, 'min_data_in_leaf': 62}. Best is trial 1 with value: 0.21752383625889085.
[I 2021-12-01 18:49:25,291] Trial 2 finished with value: 0.22507114619502672 and parameters: {'learning_rate': 0.2, 'depth': 7, 'l2_leaf_reg': 5.425489425675812, 'random_strength': 1.421700484593569, 'rsm': 0.43600668404632226, 'min_data_in_leaf': 68}. Best is trial 1 with value: 0

3_Optuna_CatBoost mae 0.077446 trained in 2714.0 seconds
* Step ensemble will try to check up to 1 model
Ensemble mae 0.073877 trained in 0.49 seconds
Skip stack because no parameters were generated.
Skip ensemble_stacked because no parameters were generated.
AutoML fit time: 21612.6 seconds
AutoML best model: Ensemble
CPU times: user 11h 14min 57s, sys: 10min 21s, total: 11h 25min 19s
Wall time: 6h 12s


In [38]:
#スコアを表示
automl_optuna.get_leaderboard()

,name,model_type,metric_type,metric_value,train_time
0,1_Optuna_LightGBM,LightGBM,mae,0.074328,6458.44
1,2_Optuna_Xgboost,Xgboost,mae,0.075588,3760.36
2,3_Optuna_CatBoost,CatBoost,mae,0.077446,2717.67
3,Ensemble,Ensemble,mae,0.073877,0.49


In [39]:
# 推論
y_pred = automl_optuna.predict(test_x)

In [40]:
test_df[TARGET] = y_pred

In [41]:
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(dir_for_output + '/test_submission.csv', index=False)